In [ ]:
import re
import json
import pymysql
import urllib.request
import urllib.parse

class query(object):
    def __init__(self):
        config = {
            'host':"10.61.2.166",
            'port':3306,
            'user':"zengyutao",
            'password':"zengyutao",
            'db':"wikidata",
            'charset':'utf8mb4'
        }
        self.notable_type_names = json.load(open("/home/jinxiaolong/zyt/kbqa_bishe/datas/zyt/all_notable_types.json","r",encoding="UTF-8"))
        self.property_names = json.load(open("/home/jinxiaolong/zyt/kbqa_bishe/datas/zyt/property_names.json","r",encoding="UTF-8"))
        self.conn = pymysql.connect(**config)
        self.cursor = self.conn.cursor()
        print("mysql connected ...")

    def id2id(self, ToColumnName, TableName, FromColumnName, OriID):
        if(OriID == None):
            return None
        sql = "select %s from %s where %s = '%s';" %(ToColumnName, TableName, FromColumnName, OriID)
        self.cursor.execute(sql)
        res = self.cursor.fetchone()
        if(res == None):
            return None
        return res[0]

    def get_index(self, qid):
        config = {
            "ToColumnName":"Table_index",
            "TableName":"entity_index",
            "FromColumnName":"Entity_id",
            "OriID":qid
        }
        return self.id2id(**config)

    def wikititle2wid(self, title):
        config = {
            "ToColumnName":"en_wid",
            "TableName":"wikititle2wid_en",
            "FromColumnName":"en_wikititle",
            "OriID":title
        }
        return self.id2id(**config)

    def wid2wikititle(self, wid):
        config = {
            "ToColumnName":"en_wikititle",
            "TableName":"wikititle2wid_en",
            "FromColumnName":"en_wid",
            "OriID":wid
        }
        return self.id2id(**config)

    def wikititle2qid(self,title):
        config = {
            "ToColumnName":"qid",
            "TableName":"qid2wikititle_en",
            "FromColumnName":"en_wikititle",
            "OriID":title
        }
        return self.id2id(**config)

    def qid2wikititle(self, qid):
        config = {
            "ToColumnName":"en_wikititle",
            "TableName":"qid2wikititle_en",
            "FromColumnName":"qid",
            "OriID":qid
        }
        return self.id2id(**config)

    def wikititle2mid(self, title):
        config = {
            "ToColumnName":"mid",
            "TableName":"mid2wikititle_en",
            "FromColumnName":"en_wikititle",
            "OriID":title
        }
        return self.id2id(**config)

    def mid2wikititle(self, mid):
        config = {
            "ToColumnName":"en_wikititle",
            "TableName":"mid2wikititle_en",
            "FromColumnName":"mid",
            "OriID":mid
        }
        return self.id2id(**config)

    def mid2qid(self, mid):
        config = {
            "ToColumnName":"qid",
            "TableName":"mid2qid",
            "FromColumnName":"mid",
            "OriID":mid
        }
        return self.id2id(**config)

    def qid2mid(self, qid):
        config = {
            "ToColumnName":"mid",
            "TableName":"mid2qid",
            "FromColumnName":"qid",
            "OriID":qid
        }
        return self.id2id(**config)
   
    def mid2wid(self, mid):
        config = {
            "ToColumnName":"en_wid",
            "TableName":"mid2wid_en",
            "FromColumnName":"mid",
            "OriID":mid
        }
        return self.id2id(**config)
    
    def wid2mid(self, wid):
        config = {
            "ToColumnName":"mid",
            "TableName":"mid2wid_en",
            "FromColumnName":"en_wid",
            "OriID":wid
        }
        return self.id2id(**config)
    
    def qid2wid_en(self, qid):
        config = {
            "ToColumnName":"en_wid",
            "TableName":"qid2wid_en",
            "FromColumnName":"qid",
            "OriID":qid
        }
        return self.id2id(**config)
        
    def qid2wid_zh(self, qid):
        config = {
            "ToColumnName":"zh_wid",
            "TableName":"qid2wid_zh",
            "FromColumnName":"qid",
            "OriID":qid
        }
        return self.id2id(**config)
    
    def wid2qid(self, wid):
        config = {
            "ToColumnName":"qid",
            "TableName":"qid2wid_en",
            "FromColumnName":"en_wid",
            "OriID":wid
        }
        res = self.id2id(**config)
        if(res == None):
            config = {
                "ToColumnName":"qid",
                "TableName":"qid2wid_zh",
                "FromColumnName":"zh_wid",
                "OriID":wid
            }
            return self.id2id(**config)
        else:
            return res 
        
    def qid2name(self, qid):
        config = {
            "ToColumnName":"Entity_name",
            "TableName":"entity_names",
            "FromColumnName":"Entity_id",
            "OriID":qid
        }
        return self.id2id(**config)
    
    def mid2name(self, mid):
        config = {
            "ToColumnName":"name",
            "TableName":"mid2name",
            "FromColumnName":"mid",
            "OriID":mid
        }
        return self.id2id(**config)
        
    def mid2type(self, mid):
        config = {
            "ToColumnName":"notable_type",
            "TableName":"mid2type",
            "FromColumnName":"mid",
            "OriID":mid
        }
        res = self.id2id(**config)
        if(res == None):
            return None
        else:
            return self.notable_type_names[res]
            
            
    def clearName(self, name):
        index1 = name.find("#")
        if(index1 == -1):
            index2 = name.find(":")
            clear_name = name[index2+1:]
        else:
            index2 = name.find(":")
            clear_name = name[index2+1:index1+1]
            index2 = name.find(":",index1)
            clear_name = clear_name + name[index2+1:]
        return clear_name
    
    ## 对得到的三元组进行处理
    ## TODO:对不同的属性值进行解释
    def processTriples(self, triples):
        result = []
        for item in triples:
            head_name = self.clearName(item[0])
            relation_name = self.property_names[item[1]]
            ## 可以跳转，为Qid或Pid
            tail_name =  item[2]
            if(item[4] == 0):
                tail_name = item[2] + ":" + self.clearName(str(self.qid2name(item[2])))
            flag = 1-item[4]
            result.append((head_name,relation_name,tail_name,flag))
        
        return result         
    
    def qid2triples(self, qid):
        index = self.get_index(qid)
        if(index == None):
            return None
        else:
            sql = "select Head_name, Relation_id, Tail, Tail_type, Tail_flag from entity_property_%s where Head_id='%s' and Relation_type='Normal';" %(index,qid)
            self.cursor.execute(sql)
            triples = self.cursor.fetchall()
            return self.processTriples(triples)
        
    ## 用于从P18:image属性中获得图片真实在线链接
    def qid2image(self, qid):
        index = self.get_index(qid)
        if(index == None):
            return None
        
        sql = "select Tail from entity_property_%s where Head_id='%s' and Relation_id='P18';" %(index, qid)
        self.cursor.execute(sql)
        res = self.cursor.fetchone()
        if(res == None):
            return None
        
        length = len("https://commons.wikimedia.org/wiki/File:")
        picname = res[0][length:]
        try:
            newurl = "https://commons.wikimedia.org/wiki/File:" + urllib.parse.quote(picname)
            page = urllib.request.urlopen(newurl)
            text = page.read().decode("UTF-8")
            pattern = re.compile('https://upload.wikimedia.org/wikipedia/commons/[^"]*')
            imageurl = pattern.findall(text)[0]
            return imageurl
        except:
            return None   
        
    def getSummaryById(self, wid):
        if(wid == None):
            return ""
        url = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&exchars=400&explaintext=true&exlimit=1&pageids=" + urllib.parse.quote(wid)
        try:
            page = urllib.request.urlopen(url)
            res = json.loads(page.read())
            return res['query']['pages'][wid]['extract']
        except:
            return ""
        
    def getImageById(self, wid):
        if(wid == None):
            return ""
        url = "https://en.wikipedia.org/w/api.php?action=query&prop=pageimages&pithumbsize=200&format=json&pageids=" + urllib.parse.quote(wid)
        try:
            page = urllib.request.urlopen(url)
            res = json.loads(page.read())
            return res['query']['pages'][wid]['thumbnail']['source']
        except:
            return ""

In [ ]:
import time
myquery = query()
myquery.mid2type("/m/0qdgz5l")

In [ ]:
import time
myquery =  query()
time1 = time.time()
print("---" + str(myquery.mid2qid("/m/03r25")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.mid2wid("/m/03r25")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.mid2wikititle("/m/03r25")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.mid2name("/m/03r25")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.mid2type("/m/03r25")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.qid2mid("Q935")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.qid2wid_en("Q935")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.qid2wid_zh("Q935")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.qid2wikititle("Q935")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.wid2mid("23680998")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.wid2qid("23680998")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.wid2wikititle("23680998")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.wikititle2mid("Hawaii")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.wikititle2qid("Hawaii")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.wikititle2wid("Hawaii")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.mid2name("/m/03r25")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.mid2type("/m/03r25")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.qid2name("Q12694")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print("---" + str(myquery.qid2name("P97")) + "---")
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

time1 = time.time()
print(myquery.qid2triples("Q26"))
time2 = time.time()
print("Time used: " + str(time2-time1) + "s")

In [ ]:
## 用于从P18:image属性中获得图片真实在线链接
import re
import urllib.request
import urllib.parse
resfile = open("./result.txt","w",encoding="UTF-8")
length = len("https://commons.wikimedia.org/wiki/File:")
with open("./images.txt","r",encoding="UTF-8") as file:
    for line in file:
        picname = line[length:-1]
        print(picname)
        try:
            newurl = "https://commons.wikimedia.org/wiki/File:" + urllib.parse.quote(picname)
            print(newurl)
            page = urllib.request.urlopen(newurl)
        except:
            resfile.write("\n")
            continue
            
        text = page.read().decode("UTF-8")
        pattern = re.compile('https://upload.wikimedia.org/wikipedia/commons/[^"]*')
        imageurl = pattern.findall(text)[0]
        resfile.write(imageurl + "\n")
resfile.close()
file.close()
print("Finished!")

In [ ]:
## 测试从mid得到相应的信息
import time
import pandas as pd
import wikipedia as wiki
time1 = time.time()
myquery = query()
answer = ["/m/0hzlz","/m/02sm8","/m/0286yf3","/m/04thp","/m/05qxk","/m/06z5s","/m/017f5","/m/025sf9q","/m/0694j","/m/025rqc3"]
result = {"mid":answer, "name":[], "type":[], "qid":[], "wid":[], "wikititle":[], "url":[]}
for mid in answer:
    result['name'].append(myquery.mid2name(mid))
    result['type'].append(myquery.mid2type(mid))
    result['qid'].append(myquery.mid2qid(mid))
    wid = myquery.mid2wid(mid)
    result['wid'].append(wid)
    result['url'].append("https://en.wikipedia.org/wiki?curid=%s" % wid)
    result['wikititle'].append(myquery.mid2wikititle(mid))

time2 = time.time()
data = pd.DataFrame(result)
print("Time used: " + str(time2-time1) + "s")
# print(data)

## image summary graph
def GetMainEntityInfo(mid):
    qid = myquery.mid2qid(mid)
    name = myquery.mid2name(mid)
    wid = myquery.qid2wid_en(qid)
    image = myquery.qid2image(qid)
    if(image == None):
        image = myquery.getImageById(wid)
    graphdata = myquery.qid2triples(qid)
    summary = myquery.getSummaryById(wid)
    return (image,summary, graphdata)      

print(GetMainEntityInfo(answer[0]))